In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
auth = pd.read_csv("../input/benfakenews/Authentic-48K.csv")
fake = pd.read_csv("../input/benfakenews/Fake-1K.csv")

In [3]:
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

In [4]:
df = pd.concat([auth[:fake.shape[0]*7], fake])
df = df.sample(frac=1).reset_index(drop=True)
len(df)

10392

In [5]:
df = df.drop('domain', axis=1)
df = df.drop('date', axis=1)
df = df.drop('category', axis=1)
df = df.drop('articleID', axis=1)

In [6]:
df['data'] = df['headline'] + ' ' + df['content']

In [7]:
df['class'] = df['label'].replace({0: 'fake', 1: 'true'})

In [8]:
df = df.drop('headline', axis=1)
df = df.drop('content', axis=1)
df = df.drop('label', axis=1)




In [9]:
df["class"].value_counts()

class
true    9093
fake    1299
Name: count, dtype: int64

In [10]:
df.dropna(inplace=True)
df["class"].value_counts()

class
true    9093
fake    1299
Name: count, dtype: int64

In [11]:
for column in df.columns:
    df[column] = df[column].str.slice(0, 100)

In [12]:
df

,data,class
0,"দেশে বিদেশে: উড়োজাহাজে প্রথম চড়ার অনুভূতি, পর্...",true
1,"ভুল চিকিৎসা, দেড় মাসে তিন প্রসূতির মৃত্যুর অভি...",true
2,ঝিনাইদহে 'বন্দুকযুদ্ধে' নিহত ১ ঝিনাইদহের কোটচা...,true
3,প্রধানমন্ত্রীর সফরসঙ্গী রিয়াজ-ফেরদৌস বিনোদন প্...,true
4,সংঘর্ষের দায়ে যুক্তরাষ্ট্রে যুবলীগের ৩ নেতাকর্...,true
...,...,...
10387,শত শত মনিরের মায়ের বদদুয়া খালেদার উপর পড়ছে: জু...,fake
10388,আম্বানিকন্যার বাগদানে তারকাদের হাট ﻿﻿﻿﻿﻿এইতো ক...,true
10389,রোমান্টিক নায়ক হিসেবে সালমান শাহ এক নিদর্শন: ঋ...,true
10390,বিএনপি-জামায়াতকে ক্ষমতার বাইরে রাখতে হবে: ইনু ...,true


In [13]:
df["class"].value_counts()

class
true    9093
fake    1299
Name: count, dtype: int64

In [14]:
df_lengths = df.applymap(lambda x: len(str(x)))

# Display the DataFrame with string lengths
print(df_lengths)

       data  class
0       100      4
1       100      4
2       100      4
3       100      4
4       100      4
...     ...    ...
10387   100      4
10388   100      4
10389   100      4
10390   100      4
10391   100      4

[10392 rows x 2 columns]


In [15]:
text = df['data'].values.tolist()
label = df['class'].tolist()

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , label, test_size=0.2, random_state=42)

In [17]:
pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 42.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 40.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 36.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.38.0-py3-none-any.whl size=25319964 sha256=71d2632f4e

In [18]:

import ktrain
from ktrain import text
MODEL_NAME = 'xlm-roberta-base'
t = text.Transformer(MODEL_NAME, maxlen=500)

In [19]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 15
	95percentile : 18
	99percentile : 20


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 15
	95percentile : 18
	99percentile : 20


In [20]:
model = t.get_classifier()

In [21]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)


In [22]:
history = learner.fit_onecycle(2e-5, 10)




begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
1386/1386 [==============================] - 699s 488ms/step - loss: 0.3504 - accuracy: 0.8542 - val_loss: 0.2274 - val_accuracy: 0.8966
Epoch 2/10
1386/1386 [==============================] - 672s 485ms/step - loss: 0.1984 - accuracy: 0.9254 - val_loss: 0.1535 - val_accuracy: 0.9380
Epoch 3/10
1386/1386 [==============================] - 672s 485ms/step - loss: 0.1350 - accuracy: 0.9530 - val_loss: 0.1135 - val_accuracy: 0.9577
Epoch 4/10
1386/1386 [==============================] - 672s 485ms/step - loss: 0.1052 - accuracy: 0.9613 - val_loss: 0.1216 - val_accuracy: 0.9505
Epoch 5/10
1386/1386 [==============================] - 672s 485ms/step - loss: 0.0912 - accuracy: 0.9688 - val_loss: 0.1481 - val_accuracy: 0.9461
Epoch 6/10
1386/1386 [==============================] - 673s 485ms/step - loss: 0.0728 - accuracy: 0.9739 - val_loss: 0.1154 - val_accuracy: 0.9634
Epoch 7/10
1386/1386 [===========================

In [23]:
learner.validate(class_names=t.get_classes()) 


65/65 [==============================] - 9s 99ms/step
              precision    recall  f1-score   support

        fake       0.85      0.87      0.86       245
        true       0.98      0.98      0.98      1834

    accuracy                           0.97      2079
   macro avg       0.92      0.92      0.92      2079
weighted avg       0.97      0.97      0.97      2079



array([[ 212,   33],
       [  36, 1798]])

In [24]:
def PlotGraph(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'g', label='Training acc')
    plt.plot(x, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'g', label='Training loss')
    plt.plot(x, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend() 

In [25]:
PlotGraph(history)

NameError: name 'plt' is not defined